In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
IMG_SIZE = 180

In [ ]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = datagen_train.flow_from_directory("../dataset/pokemon/train", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

In [ ]:
test_set = datagen_test.flow_from_directory("../dataset/pokemon/validation", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

In [ ]:
train_set.class_indices

In [ ]:
test_set.class_indices

In [ ]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

In [ ]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

In [ ]:
data_augmentation = Sequential(
    [

    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    ]
)


In [ ]:
model = Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(8,(3,3), padding='same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(len(train_set.class_indices), activation = 'softmax')
])
model.summary()


In [ ]:
opt = Adam(learning_rate=0.0001)

callback = EarlyStopping(monitor='accuracy',
                         patience=20,
                         verbose=1,
                         mode="max",
            
                         baseline=0.4,
                          restore_best_weights=True
                         )

In [ ]:
model.compile(
            optimizer=opt,
              loss = tensorflow.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=30,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                           validation_steps=STEP_SIZE_TEST,
                           batch_size=32,
                           verbose=1,
                          #  callbacks=[callback]
                           )

In [ ]:
model.save("pokemon_model.keras")